## 검색기(Retriever)
LangChain의 검색기(Retriever) 는 RAG(Retrieval-Augmented Generation) 구조에서 질문과 관련된 문서나 정보 조각을 찾아주는 핵심 모듈이다. <br>
즉, “지식 검색 엔진” 역할을 하며, LLM이 답변을 생성하기 전에 참고할 문맥(context)을 찾아준다.

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

### [1] VectorStoreRetriever

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

# 1) 문서 준비
docs = [
    Document(page_content="LangChain은 LLM 애플리케이션 개발 프레임워크입니다."),
    Document(page_content="Retriever는 문서에서 관련 정보를 검색합니다."),
    Document(page_content="FAISS는 Facebook AI가 만든 벡터 검색 라이브러리입니다.")
]

# 2) 임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 3) FAISS 벡터 DB 생성
vectorstore = FAISS.from_documents(docs, embeddings)

# 4) VectorStoreRetriever 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# 5) 질의 수행
query = "문서를 검색하는 모듈은?"
results = retriever.invoke(query)

# 6) 결과 출력
for i, doc in enumerate(results, 1):
    print(f"[결과 {i}] {doc.page_content}")

[결과 1] Retriever는 문서에서 관련 정보를 검색합니다.
[결과 2] FAISS는 Facebook AI가 만든 벡터 검색 라이브러리입니다.


In [3]:
# VectorStoreRetriever 기반 RAG 파이프라인

#  1. 필수 모듈 임포트
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

#  2. LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

#  3. Retriever (예: FAISS, Chroma 등)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

#  4. 프롬프트 정의
prompt = ChatPromptTemplate.from_template("""
아래 문서를 참고하여 질문에 답하세요.

문서 내용:
{context}

질문:
{question}
""")

# 5. RAG 파이프라인 구성
rag_chain = (
    RunnableParallel({
        "context": retriever,
        "question": RunnablePassthrough()
    })
    | prompt
    | llm
    | StrOutputParser()
)

#  6. 질의 실행
response = rag_chain.invoke("FAISS는 어떤 역할을 하나요?")
print(response)


FAISS는 벡터 검색 라이브러리로, 문서나 데이터베이스에서 유사한 벡터를 효율적으로 검색하는 역할을 합니다.


### MMR(Maximal Marginal Relevance) 검색
: MMR(Maximal Marginal Relevance) 검색은
LangChain의 VectorStoreRetriever 등에서 검색 결과의 “다양성(Diversity)”을 높이기 위한 검색 방식이다.<br>
MMR 검색은 “질문과 비슷하면서도 서로 다른 내용의 문서”를 골라주는,
중복 최소화 + 다양성 보장형 벡터 검색 방식이다. <br>
lambda_mult : 유사성과 다양성의 비율 (기본 0.5, 높을수록 유사성 우선,낮을수록 다양성 우선)

In [4]:
# MMR - 다양성 고려 (lambda_mult 작을수록 더 다양하게 추출)
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 3, 'lambda_mult': 0.15}  # lambda_mult가 0.15로 설정되어 있으므로, 관련성보다 다양성을 더 우선
)

# docs = retriever.get_relevant_documents(query)  # 1.0에서 변경됨
docs = retriever.invoke(query)
print(len(docs))
docs

3


[Document(id='cf29d58e-5f5d-44f1-b975-735b4c8669ef', metadata={}, page_content='Retriever는 문서에서 관련 정보를 검색합니다.'),
 Document(id='678dae07-121d-4cd1-8dc3-815ed395d752', metadata={}, page_content='LangChain은 LLM 애플리케이션 개발 프레임워크입니다.'),
 Document(id='27a97f6a-c7af-42da-813b-4fca12f61c1a', metadata={}, page_content='FAISS는 Facebook AI가 만든 벡터 검색 라이브러리입니다.')]

In [5]:
#  1. 모듈 임포트
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

#  2. PDF 문서 로드
loader = PyPDFLoader("유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지.pdf")
docs = loader.load()
print(f"총 {len(docs)} 페이지 로드됨")

#  3. 임베딩 모델 설정
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

#  4. FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(docs, embeddings)

#  5. MMR(Maximal Marginal Relevance) 기반 Retriever 생성
retriever = vectorstore.as_retriever(
    search_type="mmr",  
    search_kwargs={"k": 3, "lambda_mult": 0.5} 
)

#  6. 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template("""
아래의 문서를 참고하여 질문에 답하세요.

문서 내용:
{context}

질문:
{question}
""")

#  7. LLM 및 파서 설정
llm = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

#  8. RAG 파이프라인 구성
rag_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | parser
)

#  9. 질의 실행
query = "이 논문에서 리뷰 이상 탐지를 수행하는 주요 방법은 무엇인가요?"
response = rag_chain.invoke(query)

#  10. 결과 출력
print("\n질문:", query)
print("\n답변:", response)


총 16 페이지 로드됨

질문: 이 논문에서 리뷰 이상 탐지를 수행하는 주요 방법은 무엇인가요?

답변: 이 논문에서 리뷰 이상 탐지를 수행하는 주요 방법은 다음과 같이 여러 가지 기법으로 분류됩니다:

1. **비지도 학습**: F-통계를 통해 향상된 K-평균을 사용하여 리뷰를 적응적으로 클러스터링하고 비정상 그룹을 감지하여 가짜 리뷰 식별 기능을 강화합니다.

2. **지도 학습**: 리뷰 텍스트, 사용자 행동, 판매자 데이터의 기능을 사용하여 가짜 리뷰 탐지의 정확성과 효율성을 균형 있게 유지하는 강력한 성능을 보이는 DDAG-SVM 모델을 적용합니다.

3. **반지도 학습**: 일반 베이지안 모델과 수동 데이터 어노테이션 결과를 기반으로 여러 특징 조합의 효과를 평가하고, Co-training과 Tri-training이라는 두 가지 반지도 학습 전략을 도입하여 탐지 성능을 크게 향상시킵니다.

4. **텍스트 기반 분석**: 댓글의 언어적 내용과 의미적 특징을 분석하여 허위 댓글을 구별하는 주요 수단으로 사용됩니다.

5. **비판 기반 분석**: 가짜 리뷰어들이 보이는 비정상적인 평가 패턴을 분석하여 실제 사용자와의 차이를 발견하고, 이것이 탐지의 핵심 단서로 작용합니다.

6. **시간 기반 분석**: 가짜 리뷰와 실제 사용자 리뷰의 시간적 차이를 분석하여, 가짜 리뷰는 짧은 시간 내에 집중적으로 게시되는 경향이 있다는 것을 발견합니다.

7. **차트 구조 기반 분석**: 시간적 밀집성과 그래프 구조를 결합해 리뷰 그룹을 정의합니다.

8. **PU 학습 알고리즘 및 행동 밀도**: 최소한의 라벨링된 데이터를 사용하여 가짜 댓글을 감지하는 데 높은 효과를 보임.

이 기법들은 서로 보완적인 역할을 하며, 일반적으로 가짜 리뷰 또는 비정상적인 리뷰를 탐지하기 위해 활용됩니다.


In [6]:
docs = retriever.invoke(query)
print(len(docs))
docs[0]

3


Document(id='cae7c6ec-3bff-45a0-aaf5-47bae361a6e3', metadata={'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2025-10-07T19:23:49+09:00', 'moddate': '2025-10-07T19:26:28+09:00', 'source': '유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지.pdf', 'total_pages': 16, 'page': 4, 'page_label': '5'}, page_content='유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지   51\n폐된 조작된 리뷰 클러스터를 효과적으로 분리\n할 수 있으며, 탐지 시스템의 신뢰성과 견고성\n을 크게 향상시키는 데 기여한다. 따라서, 이전 \n연구에서 사용한 리뷰 분류기법의 주요 유형들\n을 고찰하였으며, 서술된 모든 방법은 <표 1>에 \n제시했다.\n2.2 극단적 및 조작된 리뷰의 식별\n온라인 사용자 리뷰는 소비자 의사결정에 강\n력한 영향을 미치는 요소로 작용하며, 그 신뢰\n성과 진정성 확보를 위한 탐지 기술의 중요성이 \n점차 강조되고 있다\n. [22]는 형용사의 극성을 판\n별하기 위해 코퍼스 기반의 비지도 학습 접근 \n방식을 제안하였다.\n이들의 방법은 텍스트 내에서 접속사(예: \n“and”, “but”)로 연결된 형용사들이 일반적으로 \n유사한 감성 극성을 지닌다는 언어학적 가정을 \n기반으로 한다. 이러한 원칙은 이후 감성어 사\n전 구축 및 감성 분석 자동화의 핵심 기초가 되\n었으며\n[23] 규칙 기반 감성 분류에서 텍스트의 \n감정 성향을 결정하는 데 널리 활용되었다. 다\n만, 해당 방식은 비속어나 신조어, 반어법 등 문\n맥에 따라 감정이 변하는 표현을 정교하게 처리\n하는 데는 한계가 존재한다.\n문서 단위에서의 감성 분류는 특히 극단적인 \n리뷰 탐지에 효과적인 접근법으로 여겨진다. \n[1]은 극단적 견해와 비극

### [2] 다중 쿼리 검색기: Multi Query Retriever
: LangChain에서 질문을 여러 방식으로 바꿔서 검색 정확도를 높이는 아주 강력한 Retriever

In [7]:
# !pip install langchain-classic

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_core.documents import Document

# 1. 문서 데이터
docs = [
    Document(page_content="LangChain은 LLM 애플리케이션 개발 프레임워크이다."),
    Document(page_content="Retriever는 문서에서 관련 정보를 검색한다."),
    Document(page_content="FAISS는 빠른 벡터 검색 라이브러리이다."),
    Document(page_content="LangChain의 검색 시스템은 Retriever를 사용한다."),
]

# 2. 벡터스토어 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embeddings)

# 3. LLM 및 기본 검색기
llm = ChatOpenAI(model="gpt-4o-mini")
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# 4. MultiQueryRetriever 생성
multi_retriever = MultiQueryRetriever.from_llm(
    retriever=base_retriever,
    llm=llm
)

# 5. 질의 실행
query = "LangChain의 문서 검색 구조는?"
results = multi_retriever.invoke(query)

# 6. 결과 출력
for i, doc in enumerate(results, 1):
    print(f"[결과 {i}] {doc.page_content}")


[결과 1] LangChain의 검색 시스템은 Retriever를 사용한다.
[결과 2] LangChain은 LLM 애플리케이션 개발 프레임워크이다.


In [9]:
# 내부 질의문 직접 보기
# queries = multi_retriever.get_queries("LangChain의 문서 검색 구조는?")  #오류 발생
# print("LLM이 생성한 내부 질의문들:")
# for i, q in enumerate(queries, 1):
#     print(f"{i}. {q}")

In [10]:
# MultiQueryRetriever 내부의 동작 흉내낸 드드
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 1. LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 2. MultiQueryRetriever 내부 프롬프트와 동일한 구조
prompt_text = """
You are an AI language model assistant. Your task is to generate
multiple alternative questions that are semantically similar
to the input question. These questions will be used to retrieve
relevant documents.

Original question: {question}

Generate {num_queries} alternative questions:
"""

prompt = PromptTemplate.from_template(prompt_text)

# 3. 안전한 사용자 정의 함수
def generate_queries(question, num_queries=3):
    """MultiQueryRetriever 내부 LLM 호출을 재현한 사용자 정의 함수"""
    formatted_prompt = prompt.format(question=question, num_queries=num_queries)
    response = llm.invoke(formatted_prompt)
    queries = [
        line.strip("-•0123456789. ").strip()
        for line in response.content.split("\n")
        if line.strip()
    ]
    return queries

# 4. 테스트 실행
queries = generate_queries("LangChain의 문서 검색 구조는?", num_queries=3)

print("LLM이 생성한 내부 질의문들:\n[원본]:LangChain의 문서 검색 구조는?")
for i, q in enumerate(queries, 1):
    print(f"{i}. {q}")


LLM이 생성한 내부 질의문들:
[원본]:LangChain의 문서 검색 구조는?
1. LangChain에서 문서 검색 시스템은 어떻게 구성되어 있나요?
2. LangChain의 문서 검색 아키텍처는 어떤 식으로 이루어져 있습니까?
3. LangChain의 문서 검색 방식은 무엇인가요?


### [3] 문맥 압축 검색기: ContextualCompressionRetriever
이 모듈은 RAG의 검색 결과를 “요약·압축”해서 전달함으로써,
LLM이 더 작은 입력으로도 중요한 정보만 이해할 수 있게 해주는 Retriever 입니다.

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document


# 1. 기본 문서
docs = [
    Document(page_content="LangChain은 LLM 애플리케이션 개발 프레임워크입니다."),
    Document(page_content="Retriever는 문서에서 관련 정보를 검색합니다."),
    Document(page_content="ContextualCompressionRetriever는 검색된 문서를 요약해 전달합니다.")
]

# 2. 벡터스토어 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embeddings)
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 3. LLM 기반 요약 압축기 생성
llm = ChatOpenAI(model="gpt-4o-mini")
compressor = LLMChainExtractor.from_llm(llm)

# 4. ContextualCompressionRetriever 생성
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# 5. 질의 실행
query = "문서 검색 결과를 요약해주는 Retriever는?"
results = compression_retriever.invoke(query)

# 6. 결과 출력
for i, doc in enumerate(results, 1):
    print(f"[요약 {i}] {doc.page_content}")


[요약 1] Retriever는 문서에서 관련 정보를 검색합니다.
[요약 2] ContextualCompressionRetriever는 검색된 문서를 요약해 전달합니다.


In [ ]:
# LangChain의 RAG 수행 예제
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# 2. PDF 문서 로드
loader = PyPDFLoader("유튜브 기반 사용자 콘텐츠에서의 리뷰 이상 탐지.pdf")
docs = loader.load()
print(f"총 {len(docs)} 페이지 로드됨")

# 3. 임베딩 모델 설정
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 4. FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(docs, embeddings)

# 5. 기본 Retriever 생성
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# 6. LLM 기반 문서 압축기 생성
llm = ChatOpenAI(model="gpt-4o-mini")
compressor = LLMChainExtractor.from_llm(llm)

# 7. ContextualCompressionRetriever 생성
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

# 8. RAG 파이프라인 구성
prompt = ChatPromptTemplate.from_template("""
아래 문서를 참고하여 질문에 답하세요.

문서 요약 내용:
{context}

질문:
{question}
""")

rag_chain = (
    RunnableParallel({"context": compression_retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

# 9. 질의 실행
query = "이 논문에서 리뷰 이상 탐지를 수행하는 주요 방법은 무엇인가요?"
response = rag_chain.invoke(query)

# 10. 결과 출력
print("\n질문:", query)
print("\n답변:", response)


총 16 페이지 로드됨
